In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
epoch_key = ('HPa', 4, 2)

In [4]:
from src.parameters import (ANIMALS, SAMPLING_FREQUENCY,
                            MULTITAPER_PARAMETERS, FREQUENCY_BANDS,
                            RIPPLE_COVARIATES)

In [5]:
from src.analysis import detect_epoch_ripples

ripple_times = detect_epoch_ripples(
    epoch_key, ANIMALS, sampling_frequency=SAMPLING_FREQUENCY)

/usr3/graduate/edeno/.conda/envs/Jadhav-2016-Data-Analysis/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [6]:
from src.data_processing import (get_LFP_dataframe, make_tetrode_dataframe)

tetrode_info = make_tetrode_dataframe(ANIMALS)[epoch_key]
tetrode_info = tetrode_info[
    ~tetrode_info.descrip.str.endswith('Ref').fillna(False)]
tetrode_info

area  depth descrip  numcells animal  day  \
animal day epoch tetrode_number                                              
HPa    4   2     1                CA1    114  riptet        10    HPa    4   
                 2                CA1    121     NaN         0    HPa    4   
                 4                CA1    117  riptet        15    HPa    4   
                 5                CA1    117  riptet         3    HPa    4   
                 6                CA1    110  riptet         0    HPa    4   
                 7                CA1    114  riptet         1    HPa    4   
                 8               iCA1    114  riptet         2    HPa    4   
                 9               iCA1    101  riptet         2    HPa    4   
                 10              iCA1     98     NaN         0    HPa    4   
                 11              iCA1    108  riptet         1    HPa    4   
                 12              iCA1    116  riptet         0    HPa    4   
                 13              iCA1    120     NaN         0    HPa    4   
                 14              iCA1    105  riptet         4    HPa    4   
                 15               PFC    116     NaN         3    HPa    4   
                 16               PFC    118     NaN         0    HPa    4   
                 17               PFC    119     NaN         0    HPa    4   
                 18               PFC    108     NaN         0    HPa    4   
                 19               PFC    126     NaN         0    HPa    4   
                 20               PFC    109     NaN         0    HPa    4   

                                 epoch  tetrode_number tetrode_id  
animal day epoch tetrode_number                                    
HPa    4   2     1                   2               1     HPa421  
                 2                   2               2     HPa422  
                 4                   2               4     HPa424  
                 5                   2               5     HPa425  
                 6                   2               6     HPa426  
                 7                   2               7     HPa427  
                 8                   2               8     HPa428  
                 9                   2               9     HPa429  
                 10                  2              10    HPa4210  
                 11                  2              11    HPa4211  
                 12                  2              12    HPa4212  
                 13                  2              13    HPa4213  
                 14                  2              14    HPa4214  
                 15                  2              15    HPa4215  
                 16                  2              16    HPa4216  
                 17                  2              17    HPa4217  
                 18                  2              18    HPa4218  
                 19                  2              19    HPa4219  
                 20                  2              20    HPa4220

In [7]:
lfps = {tetrode_key: get_LFP_dataframe(tetrode_key, ANIMALS)
        for tetrode_key in tetrode_info.index}

In [8]:
from copy import deepcopy
from functools import partial
from src.data_processing import reshape_to_segments
from src.analysis import _subtract_event_related_potential

params = deepcopy(MULTITAPER_PARAMETERS['20Hz_Resolution'])
window_of_interest = params.pop('window_of_interest')
reshape_to_trials = partial(
    reshape_to_segments,
    sampling_frequency=params['sampling_frequency'],
    window_offset=window_of_interest, concat_axis=1)

In [9]:
ripple_ERP = pd.concat({
    tetrode_info.loc[lfp_name].tetrode_id: reshape_to_trials(
        lfps[lfp_name], ripple_times).mean(axis=1)
    for lfp_name in lfps}, axis=1)

ripple_ERP

,HPa421,HPa4210,HPa4211,HPa4212,HPa4213,HPa4214,HPa4215,HPa4216,HPa4217,HPa4218,HPa4219,HPa422,HPa4220,HPa424,HPa425,HPa426,HPa427,HPa428,HPa429
time,,,,,,,,,,,,,,,,,,,
-0.4500,-6.262774,7.671533,8.029197,-3.160584,15.510949,7.255474,-11.810219,-7.627737,-4.321168,-4.708029,-2.124088,-4.138686,-9.700730,-6.065693,-5.525547,-5.518248,-6.036496,6.744526,10.985401
-0.4493,-5.890511,4.839416,5.124088,-5.182482,10.854015,4.518248,-11.277372,-8.007299,-4.138686,-3.043796,-2.343066,-4.233577,-9.715328,-6.554745,-6.138686,-4.313869,-4.948905,5.905109,8.124088
-0.4487,-6.963504,2.897810,1.846715,-6.751825,7.883212,0.437956,-11.810219,-8.160584,-4.496350,-2.167883,-1.941606,-3.532847,-8.963504,-6.437956,-6.613139,-4.000000,-5.007299,4.956204,2.897810
-0.4480,-8.532847,1.160584,-2.613139,-8.583942,5.051095,-2.781022,-11.029197,-7.284672,-3.437956,-0.729927,-0.781022,-3.489051,-8.138686,-8.124088,-8.072993,-5.583942,-7.350365,3.357664,-2.262774
-0.4473,-9.912409,0.102190,-5.693431,-9.846715,2.532847,-4.503650,-8.744526,-5.839416,-1.905109,0.802920,0.357664,-4.729927,-5.992701,-10.547445,-9.678832,-7.635036,-9.591241,1.518248,-3.226277
-0.4467,-10.131387,-0.262774,-7.021898,-10.540146,1.978102,-5.058394,-8.467153,-6.306569,-2.649635,0.335766,-0.094891,-5.255474,-5.912409,-12.467153,-10.737226,-8.700730,-11.211679,0.211679,-2.700730
-0.4460,-9.591241,-0.605839,-7.379562,-11.153285,2.795620,-5.627737,-8.423358,-6.496350,-3.401460,0.824818,-0.452555,-5.255474,-4.671533,-13.270073,-10.525547,-9.839416,-12.058394,-0.503650,-2.043796
-0.4453,-8.160584,-0.846715,-7.467153,-10.715328,5.459854,-6.189781,-7.233577,-6.963504,-3.204380,0.029197,-1.189781,-4.905109,-3.941606,-12.525547,-9.664234,-10.722628,-11.642336,-0.131387,-2.518248
-0.4447,-6.510949,-0.810219,-8.218978,-10.043796,10.080292,-5.029197,-5.167883,-7.051095,-3.364964,0.313869,-2.021898,-5.058394,-2.416058,-11.598540,-9.255474,-10.605839,-11.532847,0.810219,-1.810219


In [15]:
xr.DataArray(ripple_ERP).rename({'dim_1': 'tetrode'})

<xarray.DataArray (time: 1426, tetrode: 19)>
array([[ -6.262774,   7.671533,   8.029197, ...,  -6.036496,   6.744526,
         10.985401],
       [ -5.890511,   4.839416,   5.124088, ...,  -4.948905,   5.905109,
          8.124088],
       [ -6.963504,   2.89781 ,   1.846715, ...,  -5.007299,   4.956204,
          2.89781 ],
       ..., 
       [-21.890511,  15.80292 ,  15.591241, ..., -18.649635,   1.79562 ,
         22.058394],
       [-21.321168,  14.430657,  14.10219 , ..., -19.131387,   0.992701,
         21.116788],
       [-20.189781,  12.569343,  12.715328, ..., -19.350365,  -0.423358,
         18.59854 ]])
Coordinates:
  * time     (time) float64 -0.45 -0.4493 -0.4487 -0.448 -0.4473 -0.4467 ...
  * tetrode  (tetrode) object 'HPa421' 'HPa4210' 'HPa4211' 'HPa4212' ...

In [19]:
xr.DataArray(ripple_ERP).rename({'dim_1': 'tetrode'})

<xarray.DataArray (time: 1426, tetrode: 19)>
array([[ -6.262774,   7.671533,   8.029197, ...,  -6.036496,   6.744526,
         10.985401],
       [ -5.890511,   4.839416,   5.124088, ...,  -4.948905,   5.905109,
          8.124088],
       [ -6.963504,   2.89781 ,   1.846715, ...,  -5.007299,   4.956204,
          2.89781 ],
       ..., 
       [-21.890511,  15.80292 ,  15.591241, ..., -18.649635,   1.79562 ,
         22.058394],
       [-21.321168,  14.430657,  14.10219 , ..., -19.131387,   0.992701,
         21.116788],
       [-20.189781,  12.569343,  12.715328, ..., -19.350365,  -0.423358,
         18.59854 ]])
Coordinates:
  * time     (time) float64 -0.45 -0.4493 -0.4487 -0.448 -0.4473 -0.4467 ...
  * tetrode  (tetrode) object 'HPa421' 'HPa4210' 'HPa4211' 'HPa4212' ...